In [1]:

import pandas as pd
import openpyxl
import json
import os


In [3]:
lang = 'Hindi'
english_col = 'English copy'
allowed_values = ['x','y','z']

In [4]:
def readInputJsonAsDf(input_json_file):
    with open(input_json_file) as f:
                js = json.load(f)

    out_df = pd.DataFrame(list(js.items()),
                       columns=['Key', 'value'])        
    out_df['Key_lower'] = out_df['Key'].str.lower()
    return out_df

In [5]:
def set_values(df_row):
    if not(pd.notnull(df_row[lang])):
        pass
    else:
        df_row['value'] = df_row[lang]
    return df_row
    

In [6]:
def set_variables(df_row):
    for value in allowed_values:
        try:
            if pd.notna(df_row[value]):
                df_row[lang] = df_row[lang].replace('<'+ value + '>', df_row[value])
                df_row['key'] = df_row['key'].replace('<'+ value + '>', df_row[value])
        except:
            pass
    return df_row

In [7]:
def reformat_json(json_obj):
    json_dict = {}
    for key, value in json_obj:
        json_dict[key] = value
    return json_dict

In [8]:
def getTranslationFromSheets(locale, translation_excel_files):
    excelDf = pd.DataFrame([], columns=['Key', lang])
    for excel_file_name in translation_excel_files:
#         print("---------------",excel_file_name, "------------------")
        excel = pd.ExcelFile(locale + '/' + excel_file_name)

        count = 0
        for sheetName in excel.sheet_names:
            sheet = excel.parse(sheet_name = sheetName, header=1)
            if(len(sheet.columns) == 0):
                continue
#             print(sheetName, sheet.columns)
            FORMAT = [english_col,lang]
            for value in allowed_values:
                if value in sheet.columns:
                    FORMAT.append(value)
            filteredSheet = sheet[FORMAT]
            sheet_no_na = filteredSheet.dropna(subset = [english_col, lang], inplace=False)
            sheet_new = sheet_no_na.rename(columns = {english_col: 'Key'}, inplace=False)
            count += sheet_new.count()
            excelDf = pd.concat([excelDf, sheet_new], axis=0)
            
    excelDf = excelDf.apply(set_variables, axis=1)
    return excelDf

In [18]:
def getAllTranslations():
#     ('ta','Tamil')('te','Telugu') ('pa','Punjabi'),('mr','Marathi'),('be','Bengali')
    locales = [('hi','Hindi')]
    allTranslations = pd.DataFrame()
    for locale, language in locales:
        global lang
        lang = language
        path_to_xl = './'+ locale
        excel_files = sorted(os.listdir(path_to_xl))
        translation_excel_files = [pos_json for pos_json in excel_files if pos_json.endswith('.xlsx') and not pos_json.startswith('~')]
        
        excelDf = getTranslationFromSheets(locale, translation_excel_files)
        out_df = readInputJsonAsDf('./../../../crowdsource-ui/locales/{locale}.json'.format(locale=locale))
            
        excelDf['Key_lower'] = excelDf['Key'].str.lower().str.strip()
        out_df['Key_lower'] = out_df['Key'].str.lower().str.strip()
        
        excelDf = excelDf.drop_duplicates(subset=['Key_lower'], keep='last')
        
        excelDf_dropped = excelDf.drop_duplicates(subset=['Key_lower'], keep='first')
        out_df_dropped = out_df.drop_duplicates(subset=['Key'], keep='first')
#         print(excelDf_dropped[excelDf_dropped['Key'].duplicated()])
#         allTranslations[locale+'_key'] = excelDf_dropped['Key']
#         allTranslations[locale+'_value']=excelDf_dropped[language]
        
        
        merged_df = pd.merge(excelDf_dropped, out_df_dropped, on="Key_lower", how='right')

        merged_df = merged_df.apply(set_values, axis = 1)

        select_columns = ['Key_y', 'value']

        filtered_merged_df = merged_df[select_columns]

        final_df = filtered_merged_df.drop_duplicates(subset='Key_y', keep='first', inplace=False)

        jsonFile = final_df.to_json(orient='values')
        jsonFile = json.loads(jsonFile)
        
        final_final_json = reformat_json(jsonFile)
        
        output_json_file = locale + '.json'

        with open(output_json_file, 'w') as f:
            f.write(json.dumps(final_final_json, indent = 4, ensure_ascii=False))
    return allTranslations


In [19]:
all = getAllTranslations()
all.head()

""


In [ ]:
## Test Block

In [ ]:
locale = 'hi'
path_to_xl = './'+ locale
excel_files = sorted(os.listdir(path_to_xl))
translation_excel_files = [pos_json for pos_json in excel_files if pos_json.endswith('.xlsx') and not pos_json.startswith('~')]

excelDf = getTranslationFromSheets(locale, translation_excel_files)
out_df = readInputJsonAsDf('./../../../crowdsource-ui/locales/{locale}.json'.format(locale=locale))

excelDf['Key_lower'] = excelDf['Key'].str.lower().str.strip()
out_df['Key_lower'] = out_df['Key'].str.lower().str.strip()

excelDf = excelDf.drop_duplicates(subset=['Key_lower'], keep='last')

excelDf_dropped = excelDf.drop_duplicates(subset=['Key_lower'], keep='first')
out_df_dropped = out_df.drop_duplicates(subset=['Key'], keep='first')


merged_df = pd.merge(excelDf_dropped, out_df_dropped, on="Key_lower", how='right')

merged_df = merged_df.apply(set_values, axis = 1)

select_columns = ['Key_y', 'value']

filtered_merged_df = merged_df[select_columns]

final_df = filtered_merged_df.drop_duplicates(subset='Key_y', keep='first', inplace=False)

jsonFile = final_df.to_json(orient='values')
jsonFile = json.loads(jsonFile)

final_final_json = reformat_json(jsonFile)

output_json_file = locale + '.json'

# with open(output_json_file, 'w') as f:
#     f.write(json.dumps(final_final_json, indent = 4, ensure_ascii=False))


In [ ]:
import re
count = 0
checkCount = 0
for key, value in jsonFile:
    x = re.search("^[\u0020-\u007F]+$", value)
    count += 1
    if x:
      checkCount += 1
print(count, checkCount)

In [11]:
def validateAndPrintResult(locale,lang, translation_excel_files):
    for excel_file_name in translation_excel_files:
        print("---------------",excel_file_name, "------------------")
        excel = pd.ExcelFile(locale + '/' + excel_file_name)
        for sheetName in excel.sheet_names:
            sheet = excel.parse(sheet_name = sheetName, header=1)
            FORMAT = ['English copy',lang]
            for value in allowed_values:
                if value in sheet.columns:
                    FORMAT.append(value)
            if 'English copy' not in sheet.columns:
                print('{} does not have English copy or C in copy is in caps'.format(sheetName))
            if lang not in sheet.columns:
                print('{} does not have {} column or has some hidden sheet(If so, unhide and delete it).'.format(sheetName, lang))

In [15]:
locale = 'gu'
path_to_xl = './'+ locale
excel_files = sorted(os.listdir(path_to_xl))
translation_excel_files = [pos_json for pos_json in excel_files if pos_json.endswith('.xlsx') and not pos_json.startswith('~')]
validateAndPrintResult(locale,'Gujarati', translation_excel_files)


--------------- a.xlsx ------------------
--------------- b.xlsx ------------------
--------------- c.xlsx ------------------
--------------- d.xlsx ------------------
--------------- e.xlsx ------------------
--------------- f.xlsx ------------------
--------------- g.xlsx ------------------
--------------- h.xlsx ------------------
--------------- i.xlsx ------------------
--------------- j.xlsx ------------------
--------------- k.xlsx ------------------
--------------- l.xlsx ------------------
--------------- m.xlsx ------------------
--------------- n.xlsx ------------------
--------------- o.xlsx ------------------
Delta 2 does not have Gujarati column or has some hidden sheet(If so, unhide and delete it).
--------------- p.xlsx ------------------
Delta does not have Gujarati column or has some hidden sheet(If so, unhide and delete it).
--------------- q.xlsx ------------------
Delta 2 does not have Gujarati column or has some hidden sheet(If so, unhide and delete it).
---------